In [1]:
import math

import torch

from script.NeuralNets.Networks import SequentialNN, ICNN, ICNNApproxMax, ICNNLogical
from script.settings import device, data_type
import script.DHOV.MultiDHOV as multidhov
from script.Verification.Verifier import SingleNeuronVerifier, MILPVerifier, DHOVVerifier
import gurobipy as grp
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, ToTensor, Normalize
from script.NeuralNets.ICNNFactory import ICNNFactory

In [2]:
def add_max_constr(model, neuron_name):
    neuron_var = model.getVarByName(neuron_name)
    model.setObjective(neuron_var, grp.GRB.MAXIMIZE)

def add_min_constr(model, neuron):
    neuron_var = model.getVarByName(neuron)
    model.setObjective(neuron_var, grp.GRB.MINIMIZE)

In [3]:
def optimize_model(model, neuron_name):
    model.update()
    model.optimize()
    if model.Status == grp.GRB.OPTIMAL:
        print("opt value: {}".format(model.getVarByName(neuron_name).getAttr("x")))

In [4]:
def icnn_model(icnn, nn, input_x, eps, layer_index, from_neuron, to_neuron, print_log=False):
    m = grp.Model()
    if not print_log:
        m.Params.LogToConsole = 0

    input_flattened = torch.flatten(input_x)
    bounds_affine_out, bounds_layer_out = nn.calculate_box_bounds(
        [input_flattened.add(-eps), input_flattened.add(eps)])

    parameter_list = list(nn.parameters())

    input_size = len(parameter_list[2*layer_index])
    lb = bounds_layer_out[layer_index][0].detach().cpu().numpy()
    ub = bounds_layer_out[layer_index][1].detach().cpu().numpy()
    in_var = m.addMVar(input_size, lb=lb, ub=ub, name="icnn_var")

    low = bounds_layer_out[layer_index][0][from_neuron: to_neuron]
    up = bounds_layer_out[layer_index][1][from_neuron: to_neuron]
    constraint_bounds_affine_out, constraint_bounds_layer_out = icnn.calculate_box_bounds([low, up])
    icnn.add_max_output_constraints(m, in_var[from_neuron: to_neuron], constraint_bounds_affine_out, constraint_bounds_layer_out)

    return m

In [5]:
"""W1 = [1. 1.; 1. -1.]
    b1 = [0., 0.]
    W2 = [1. 1.; 1. -1.]
    b2 = [-0.5, 0.]
    W3 = [-1. 1.; 1. 1.]
    b3 = [3., 0.] """

"""nn = SequentialNN([2, 2, 2, 2])

with torch.no_grad():
    parameter_list = list(nn.parameters())
    parameter_list[0].data = torch.tensor([[1, 1], [1, -1]], dtype=data_type).to(device)
    parameter_list[1].data = torch.tensor([0, 0], dtype=data_type).to(device)
    parameter_list[2].data = torch.tensor([[1, 1], [1, -1]], dtype=data_type).to(device)
    parameter_list[3].data = torch.tensor([-0.5, 0], dtype=data_type).to(device)
    parameter_list[4].data = torch.tensor([[-1, 1], [1, 1]], dtype=data_type).to(device)
    parameter_list[5].data = torch.tensor([3, 0], dtype=data_type).to(device)

test_image = torch.tensor([[0, 0]], dtype=data_type).to(device)"""

"""transform = Compose([ToTensor(),
                         Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
                        )

training_data = CIFAR10(root="../../cifar", train=True, download=True, transform=transform)
images, labels = training_data.__getitem__(0)
test_image, test_label = torch.unsqueeze(images, 0).to(dtype=data_type).to(device), torch.unsqueeze(
    torch.tensor(labels), 0).to(dtype=data_type).to(device)

nn = SequentialNN([32 * 32 * 3, 1024, 512, 10])
nn.load_state_dict(torch.load("../../cifar_fc.pth", map_location=torch.device(device)), strict=False)
parameter_list = list(nn.parameters())"""

nn = SequentialNN([50, 50, 50, 7])
test_image = torch.zeros((1, 50), dtype=data_type).to(device)
parameter_list = list(nn.parameters())

In [6]:
eps = 0.01
layer_index = 1
neuron_index = 0
neuron_name = "relu_var{}[{}]".format(2*layer_index, neuron_index)

Test for SNV

In [8]:
snv_verifier = SingleNeuronVerifier(nn, test_image, eps, print_log=False)
snv_verifier.generate_constraints_for_net()
snv_model = snv_verifier.model
snv_model.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-12


In [9]:
snv_copy = snv_model.copy()
snv_copy.Params.LogToConsole = 0
add_min_constr(snv_copy, neuron_name)
optimize_model(snv_copy, neuron_name)

opt value: 0.0


In [10]:
snv_copy = snv_model.copy()
snv_copy.Params.LogToConsole = 0
add_max_constr(snv_copy, neuron_name)
optimize_model(snv_copy, neuron_name)

opt value: 1.2794941959311528


Test for MILP

In [23]:
neuron_name = "relu_var{}[{}]".format(2*layer_index, 0)
milp_verifier = MILPVerifier(nn, test_image, eps, print_log=False)
milp_verifier.generate_constraints_for_net(until_layer_neuron=[layer_index, neuron_index])
milp_model = milp_verifier.model
milp_model.update()

"""all_var = milp_model.getVars()
for var in all_var:
    print(var)"""


'all_var = milp_model.getVars()\nfor var in all_var:\n    print(var)'

In [24]:
milp_copy = milp_model.copy()
milp_copy.Params.LogToConsole = 0
add_min_constr(milp_copy, neuron_name)
optimize_model(milp_copy, neuron_name)

opt value: 0.0


In [25]:
milp_copy = milp_model.copy()
milp_copy.Params.LogToConsole = 0
add_max_constr(milp_copy, neuron_name)
optimize_model(milp_copy, neuron_name)

opt value: 0.5620409095272554


Test for DHOV

In [7]:
group_size = 3
icnn_factory = ICNNFactory("logical", [5, 5, 1], force_positive_init=False, with_two_layers=False,
                               init_scaling=10, init_all_with_zeros=False)
#icnn_factory = ICNNFactory("standard", [5, 5, 1])

icnns, all_group_indices, fixed_neuron_per_layer_lower, fixed_neuron_per_layer_upper, bounds_affine_out, bounds_layer_out = \
    multidhov.start_verification(nn, test_image, icnn_factory, group_size, eps=eps, icnn_epochs=10, icnn_batch_size=1000,
                                 sample_count=10, sample_new=True, use_over_approximation=True, break_after=152,
                                 sample_over_input_space=False, sample_over_output_space=True, use_icnn_bounds=True, use_fixed_neurons=True,
                                 force_inclusion_steps=0, preemptive_stop=False, even_gradient_training=False,
                                 keep_ambient_space=True, data_grad_descent_steps=0, train_outer=False,
                                 should_plot="none", optimizer="SdLBFGS", init_network=True, adapt_lambda="none")




approximation of layer: 0
    number of fixed neurons for current layer: 33
Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-12
    layer progress, group 1 of 6 
        time for training: 0.08463048934936523


C:\Users\Ufuk\Documents\Programming\ICNN_verification\script\Optimizer\sdlbfgs.py:83: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ..\torch\csrc\utils\python_arg_parser.cpp:1420.)
  p.data.add_(step_size, update[offset:offset + numel].view_as(p.data))


        actual verification time 0.01563429832458496
        time for verification: 0.21597766876220703
        lower 0.0
        upper 0.03829004615545273
        lower 0.0
        upper 0.014630567282438278
        lower 0.0
        upper 0.055084407329559326
    layer progress, group 2 of 6 
        time for training: 0.062050580978393555
        actual verification time 0.00201416015625
        time for verification: 0.16524648666381836
        lower 0.0
        upper 0.02521830129817282
        lower 0.0
        upper 0.006415143609046936
        lower 0.0
        upper 0.04361312035566126
    layer progress, group 3 of 6 
        time for training: 0.05032706260681152
        actual verification time 0.010158300399780273
        time for verification: 0.16325163841247559
        lower 0.0
        upper 0.026150763034820557
        lower 0.0
        upper 0.031326234340667725
        lower 0.0
        upper 0.0026340633630752563
    layer progress, group 4 of 6 
        time for t

In [9]:
print(bounds_affine_out)

[[tensor([ 0.2088,  0.0196, -0.0026,  ..., -1.3163, -0.3677, -0.2236],
       grad_fn=<AddBackward0>), tensor([ 0.7791,  0.5886,  0.5620,  ..., -0.7273,  0.2118,  0.3357],
       grad_fn=<AddBackward0>)]]


In [14]:
input_flattened = torch.flatten(test_image)
bounds_affine_out, bounds_layer_out = nn.calculate_box_bounds([input_flattened.add(-eps), input_flattened.add(eps)])
print("bounds affine out \n {}".format(bounds_affine_out))

bounds affine out 
 [[tensor([ 0.2088,  0.0196, -0.0026,  ..., -1.3163, -0.3677, -0.2236],
       grad_fn=<AddBackward0>), tensor([ 0.7791,  0.5886,  0.5620,  ..., -0.7273,  0.2118,  0.3357],
       grad_fn=<AddBackward0>)], [tensor([-2.3678, -2.2506, -1.4631, -1.5480, -1.6997, -2.3157, -2.0312, -2.1657,
        -2.4451, -2.1094, -2.5736, -1.5114, -1.7204, -1.6585, -1.9900, -2.2936,
        -1.8326, -1.9801, -2.0940, -1.9852, -2.4245, -2.3254, -1.7885, -2.0473,
        -1.7248, -2.0310, -2.1796, -1.5130, -2.1948, -1.2174, -2.3810, -1.6069,
        -2.2250, -1.0840, -2.1734, -1.6119, -2.2030, -2.1282, -2.2584, -2.3525,
        -1.9250, -2.2268, -1.9972, -1.8989, -2.1081, -1.8777, -2.4823, -2.0515,
        -2.2419, -2.3808, -2.0566, -2.1669, -1.8904, -2.0341, -2.4128, -2.7350,
        -2.8463, -1.6933, -2.4030, -2.0562, -1.9528, -2.1218, -2.4711, -2.3628,
        -2.2642, -2.0658, -1.8225, -1.7588, -2.5546, -1.6129, -1.8245, -2.1414,
        -1.9952, -1.9148, -2.2306, -2.0037, -2.5366, -

In [15]:
dhov_model = icnn_model(icnns[layer_index][0], nn, test_image, eps, layer_index, 0, group_size, print_log=False)
dhov_model.update()

In [16]:
icnn_neuron_name = "icnn_var[{}]".format(neuron_index)

In [17]:
dhov_copy = dhov_model.copy()
dhov_copy.Params.LogToConsole = 0
add_min_constr(dhov_copy, icnn_neuron_name)
optimize_model(dhov_copy, icnn_neuron_name)

opt value: 0.0


In [18]:
dhov_copy = dhov_model.copy()
dhov_copy.Params.LogToConsole = 0
add_max_constr(dhov_copy, icnn_neuron_name)
optimize_model(dhov_copy, icnn_neuron_name)

opt value: 2.198498487472534
